In [ ]:
from michigrad.engine import Value

# dataset XOR
dataset = [
    ([Value(0), Value(0)], Value(0)), # 0 xor 0 = 0
    ([Value(0), Value(1)], Value(1)), # 0 xor 1 = 1
    ([Value(1), Value(0)], Value(1)), # 1 xor 0 = 1
    ([Value(1), Value(1)], Value(0)), # 1 xor 1 = 0
]

In [ ]:
from michigrad.nn import MLP

# Ahora uso un MLP con una capa oculta de 2 neuronas y una neurona de salida.
# Arquitectura: 2 -> 2 -> 1
# La capa oculta usa ReLU (por defecto), la salida es lineal.
model = MLP(2, [2, 1])

print(model)

In [ ]:
from michigrad.visualize import show_graph

params = model.parameters()
lr = 0.1   # learning rate

# Entrenamiento
for epoch in range(200):

    # Reseteo loss
    loss = Value(0)

    # Forward: Cálculo de L con suma de errores cuadrados
    for x, y in dataset:
        yhat = model(x)
        loss = loss + (y - yhat)**2

    # Pongo todos los gradientes en cero
    model.zero_grad()

    # Backpropagation
    loss.backward()

    if epoch == 0:
      print("Grafo después del backward:")
      dot = show_graph(loss, rankdir="TB", format="png")
      # para que me lo imprima Jupyter a pesar de no ser la última línea
      display(dot)

    # Descenso del gradiente (actualiza parámetros) 
    for p in params:
        p.data -= lr * p.grad

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: loss = {loss.data}")

# Predicciones finales sobre la tabla XOR
print("\nPredicciones del modelo entrenado:")
for x, y in dataset:
    yhat = model(x)
    salida = 1 if yhat.data > 0.5 else 0
    print(f"Entrada { [v.data for v in x] } -> salida cruda = {yhat.data:.4f} | salida binaria = {salida} | esperado = {y.data}")

### Interpretación de los resultados del modelo entrenado

En este ejercicio rehice el problema de la función XOR usando ahora una red neuronal construida con **Michigrad**. A diferencia del caso anterior, donde trabajaba con una sola neurona lineal, acá utilicé un MLP con arquitectura 2–2–1: dos neuronas en una capa oculta con activación ReLU y una neurona de salida lineal. El XOR es un ejemplo clásico de problema no linealmente separable, por lo que un modelo puramente lineal no puede resolverlo. La capa oculta introduce una transformación no lineal del espacio de entrada; cada neurona aprende a activar en distintas regiones del plano definido por las dos entradas. Luego, la neurona de salida combina esas activaciones para aproximar la tabla de verdad del XOR. Después del entrenamiento, el modelo devuelve valores cercanos a 0 y 1 en los cuatro patrones y, aplicando un umbral sencillo, logra clasificar correctamente todos los casos.